In [1]:
# Import necessary libraries for environment, agents, HTTP requests, data modeling, and display
import os
import asyncio
import requests
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from typing import Dict
from IPython.display import display, Markdown
from twilio.rest import Client  # Import Twilio client for WhatsApp

# Load environment variables from .env file
# override=True ensures existing variables are updated if present
load_dotenv(override=True)

# Retrieve WhatsApp credentials from environment variables
WHATSAPP_ACCOUNT_SID = os.getenv('WHATSAPP_ACCOUNT_SID')
WHATSAPP_AUTH_TOKEN = os.getenv('WHATSAPP_AUTH_TOKEN')
WHATSAPP_FROM_NUMBER = os.getenv('WHATSAPP_FROM_NUMBER')

if not all([WHATSAPP_ACCOUNT_SID, WHATSAPP_AUTH_TOKEN, WHATSAPP_FROM_NUMBER]):
    raise ValueError("WhatsApp API credentials (WHATSAPP_ACCOUNT_SID, WHATSAPP_AUTH_TOKEN, WHATSAPP_FROM_NUMBER) not found in environment variables. Please add them to your .env file.")

# Define constant for number of searches to perform
HOW_MANY_SEARCHES = 3

print("Imports and initial setup complete.")

Imports and initial setup complete.


In [2]:
# Define instructions for the search agent to summarize web results
SEARCH_AGENT_INSTRUCTIONS = """You are a research assistant. Given a search term, you search the web for that term and produce a concise summary of the results. The summary must be 2-3 paragraphs and less than 300 words. Capture the main points. Write succinctly, no need to have complete sentences or good grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the essence and ignore any fluff. Do not include any additional commentary other than the summary itself."""

# Define instructions for the planner agent to generate search terms
PLANNER_AGENT_INSTRUCTIONS = f"""You are a helpful research assistant. Given a query, come up with a set of web searches to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."""

# Define instructions for the writer agent to create a detailed report
WRITER_AGENT_INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

# Define instructions for the WhatsApp agent to send formatted messages
WHATSAPP_AGENT_INSTRUCTIONS = """You are able to send a research report to a WhatsApp number based on a detailed report.
You will be provided with a detailed report and a WhatsApp number. You should use your tool to send the report in a well-formatted manner suitable for WhatsApp messaging.

For WhatsApp formatting:
- Break long content into multiple messages if needed
- Use emojis strategically for better readability
- Use *bold* for emphasis where appropriate
- Keep messages mobile-friendly and easy to read
- Add appropriate subject lines or headers for each section"""

print("Agent instructions defined.")

Agent instructions defined.


In [3]:
# Define a Pydantic model for a single web search item
class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")

# Define a Pydantic model for the web search plan
class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

# Define a Pydantic model for the report output
class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(description="The final report")
    follow_up_questions: list[str] = Field(description="Suggested topics to research further")

print("Pydantic models for structured outputs defined.")

Pydantic models for structured outputs defined.


In [4]:
# Create a search agent to perform web searches and summarize results
search_agent = Agent(
    name="Search agent",
    instructions=SEARCH_AGENT_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

# Create a planner agent to generate a structured web search plan
planner_agent = Agent(
    name="PlannerAgent",
    instructions=PLANNER_AGENT_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

# Create a writer agent to produce a detailed report
writer_agent = Agent(
    name="WriterAgent",
    instructions=WRITER_AGENT_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

print("Search, Planner, and Writer agents defined.")

Search, Planner, and Writer agents defined.


In [5]:
# Define a tool to send messages to WhatsApp via Twilio
@function_tool
def send_whatsapp_message(to_number: str, message_content: str) -> Dict[str, str]:
    """
    Sends a message to a WhatsApp number using the Twilio API.
    Handles long content by splitting it into multiple messages if needed.
    
    Args:
        to_number (str): The recipient's WhatsApp number (e.g., "+1234567890").
                         Must include the country code and be formatted correctly.
        message_content (str): The content of the message to send.
    Returns:
        Dict[str, str]: A dictionary indicating the status of the message sending.
    """
    if not all([WHATSAPP_ACCOUNT_SID, WHATSAPP_AUTH_TOKEN, WHATSAPP_FROM_NUMBER]):
        return {"status": "error", "message": "WhatsApp API credentials not found in environment variables."}

    # WhatsApp message length limit (approximately 4096 characters)
    MAX_MESSAGE_LENGTH = 4000
    
    try:
        client = Client(WHATSAPP_ACCOUNT_SID, WHATSAPP_AUTH_TOKEN)
        
        # Split long content into multiple messages if needed
        if len(message_content) <= MAX_MESSAGE_LENGTH:
            # Send as single message
            message = client.messages.create(
                from_=f'whatsapp:{WHATSAPP_FROM_NUMBER}',
                body=message_content,
                to=f'whatsapp:{to_number}'
            )
            return {"status": "success", "sid": message.sid, "message": f"Message sent to WhatsApp successfully. SID: {message.sid}"}
        else:
            # Split into multiple messages
            content_chunks = []
            current_chunk = ""
            for line in message_content.splitlines(True):
                if len(current_chunk) + len(line) <= MAX_MESSAGE_LENGTH:
                    current_chunk += line
                else:
                    if current_chunk:
                        content_chunks.append(current_chunk)
                    current_chunk = line
            if current_chunk:
                content_chunks.append(current_chunk)
            
            # Send multiple messages
            message_sids = []
            for i, chunk in enumerate(content_chunks):
                header = f"📄 Research Report (Part {i+1}/{len(content_chunks)})\n\n" if i > 0 else "📄 Research Report\n\n"
                message_body = header + chunk
                
                message = client.messages.create(
                    from_=f'whatsapp:{WHATSAPP_FROM_NUMBER}',
                    body=message_body,
                    to=f'whatsapp:{to_number}'
                )
                message_sids.append(message.sid)
            
            return {"status": "success", "sids": message_sids, "message": f"Report sent in {len(content_chunks)} WhatsApp messages successfully."}
            
    except Exception as e:
        return {"status": "error", "message": f"Failed to send WhatsApp message: {e}"}

In [6]:
# Create a WhatsApp agent to send formatted messages using the Twilio tool
whatsapp_agent = Agent(
    name="WhatsApp Agent",
    instructions=WHATSAPP_AGENT_INSTRUCTIONS,
    tools=[send_whatsapp_message],
    model="gpt-4o-mini",
)

print("WhatsApp Agent defined.")

WhatsApp Agent defined.


In [7]:
# Define a function to plan web searches for a query
async def plan_searches(query: str):
    """Use the planner_agent to plan which searches to run for the query"""
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

# Define a function to perform searches concurrently
async def perform_searches(search_plan: WebSearchPlan):
    """Call search() for each item in the search plan concurrently"""
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

# Define a function to execute a single search
async def search(item: WebSearchItem):
    """Use the search agent to run a web search for each item in the search plan"""
    input_message = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input_message)
    return result.final_output

# Define a function to write a report based on search results
async def write_report(query: str, search_results: list[str]):
    """Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input_message = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input_message)
    print("Finished writing report")
    return result.final_output

print("Orchestration functions for planning, searching, and writing report defined.")

Orchestration functions for planning, searching, and writing report defined.


In [8]:
# Define a function to send a report to WhatsApp
async def send_report_to_whatsapp(report: ReportData, whatsapp_number: str):
    """Use the whatsapp agent to send a WhatsApp message with the report"""
    print("Preparing WhatsApp message...")
    input_message = f"Send this research report to WhatsApp number {whatsapp_number}:\n\n{report.markdown_report}"
    result = await Runner.run(whatsapp_agent, input_message)
    print(f"WhatsApp message status: {result.final_output}")
    return report

print("Orchestration function for sending report to WhatsApp defined.")

Orchestration function for sending report to WhatsApp defined.


In [9]:
# Define the research query and WhatsApp number
query = "The impact of quantum computing on cryptography by 2030"
whatsapp_number = "+4915222350056"  # Replace with your actual WhatsApp number

print(f"Starting research for query: \"{query}\"")
print(f"Report will be sent to WhatsApp: {whatsapp_number}")

# Execute the research workflow
with trace("Research trace"):
    print("Starting research process...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_report_to_whatsapp(report, whatsapp_number)
    print("Hooray! Research and WhatsApp notification complete.")

# Display the final report in the notebook
display(Markdown(report.markdown_report))
print(f"Short Summary: {report.short_summary}")
print(f"Follow-up Questions: {report.follow_up_questions}")

print("\nWorkflow execution complete. Check your WhatsApp for the report!")
print("Remember to check the trace at https://platform.openai.com/traces for detailed execution steps.")

Starting research for query: "The impact of quantum computing on cryptography by 2030"
Report will be sent to WhatsApp: +4915222350056
Starting research process...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Preparing WhatsApp message...
WhatsApp message status: The research report titled "*The Impact of Quantum Computing on Cryptography by 2030*" has been successfully sent to the WhatsApp number +4915222350056. If you need any further assistance or details, feel free to ask! 📩
Hooray! Research and WhatsApp notification complete.


# The Impact of Quantum Computing on Cryptography by 2030

## Table of Contents

1. [Introduction](#introduction)  
2. [Understanding Quantum Computing](#understanding-quantum-computing)  
   1. [Defining Quantum Computing](#defining-quantum-computing)  
   2. [Quantum Mechanics Basics](#quantum-mechanics-basics)  
3. [Current Cryptographic Systems](#current-cryptographic-systems)  
   1. [Symmetric vs. Asymmetric Cryptography](#symmetric-vs-asymmetric-cryptography)  
   2. [Key Cryptographic Algorithms Vulnerable to Quantum Computing](#key-cryptographic-algorithms-vulnerable-to-quantum-computing)  
4. [The Risks Posed by Quantum Computing](#the-risks-posed-by-quantum-computing)  
   1. [Shor’s Algorithm and Its Implications](#shors-algorithm-and-its-implications)  
   2. [Potential Consequences of Cryptographic Failure](#potential-consequences-of-cryptographic-failure)  
5. [Call to Action: Preparing for Quantum Threats](#call-to-action-preparing-for-quantum-threats)  
   1. [Recommendations from Cyber Security Authorities](#recommendations-from-cyber-security-authorities)  
   2. [Strategies for Transitioning to Quantum-Resistant Cryptography](#strategies-for-transitioning-to-quantum-resistant-cryptography)  
6. [Post-Quantum Cryptography Developments](#post-quantum-cryptography-developments)  
   1. [NIST's Role in Developing New Standards](#nist-role-in-developing-new-standards)  
   2. [Emerging Post-Quantum Algorithms](#emerging-post-quantum-algorithms)  
7. [Case Studies and Real-World Applications](#case-studies-and-real-world-applications)  
   1. [Financial Sector Initiatives](#financial-sector-initiatives)  
   2. [Telecommunications Advancements](#telecommunications-advancements)  
8. [Conclusion](#conclusion)  
9. [Follow-Up Questions](#follow-up-questions)  

## Introduction  
Quantum computing is on the rise, with predictions of operational large-scale quantum computers by 2030. This technological shift poses a considerable risk to current cryptographic systems, designed primarily under classical computing paradigms. As organizations prepare for this disruption, understanding the fundamental principles of quantum computing and their implications for cryptography becomes essential.

## Understanding Quantum Computing  
### Defining Quantum Computing  
Quantum computing leverages the principles of quantum mechanics to process information in ways unimaginable through classical computing. Unlike classical bits, which represent data as either 0 or 1, quantum bits, or qubits, can exist in numerous states simultaneously, allowing for complex calculations to be performed exponentially faster.

### Quantum Mechanics Basics  
Quantum mechanics introduces phenomena such as superposition and entanglement. Superposition allows qubits to be in multiple states at once, while entanglement connects qubits such that the state of one can instantaneously affect the state of another, regardless of distance. These principles underpin the power of quantum computers.

## Current Cryptographic Systems  
### Symmetric vs. Asymmetric Cryptography  
Cryptography generally includes two types:  
- **Symmetric Encryption:** Utilizes the same key for both encryption and decryption (e.g., AES).
- **Asymmetric Encryption:** Employs a pair of keys (public and private), mainly represented by algorithms like RSA and ECC, which are susceptible to quantum attacks.

### Key Cryptographic Algorithms Vulnerable to Quantum Computing  
The security of many commonly adopted encryption techniques relies on the difficulty of mathematical problems, such as:
- **Prime Factorization:** A cornerstone of RSA encryption.
- **Discrete Logarithms:** A fundamental part of ECC.
These algorithms are particularly vulnerable to quantum algorithms, especially Shor’s algorithm, which can compute these problems in polynomial time.

## The Risks Posed by Quantum Computing  
### Shor’s Algorithm and Its Implications  
Shor's algorithm, developed by mathematician Peter Shor, demonstrates that quantum computers could efficiently factor large integers, breaking RSA encryption and potentially compromising any systems relying on such encryption by 2030.

### Potential Consequences of Cryptographic Failure  
The implications of failing to transition to quantum-resistant systems are severe, including  
- Unauthorized access to sensitive information (financial, medical, personal).
- Risk of data integrity and authenticity compromised.  
- Increased cyber risks given the capability of quantum computers to undermine trust in digital communications.

## Call to Action: Preparing for Quantum Threats  
### Recommendations from Cyber Security Authorities  
Organizations worldwide, including the UK’s National Cyber Security Centre (NCSC) and the National Institute of Standards and Technology (NIST) in the U.S., recommend proactive measures:
- Begin transitioning to post-quantum cryptography as early as 2028.
- Regularly update cryptographic infrastructure and stay informed about ongoing advancements in quantum technology.

### Strategies for Transitioning to Quantum-Resistant Cryptography  
Organizations should  
- Conduct assessments of their current cryptographic systems and their vulnerabilities.
- Start pilot projects to understand the implementation challenges of new algorithms.
- Collaborate with industry peers to share best practices and develop a common roadmap toward quantum resilience.

## Post-Quantum Cryptography Developments  
### NIST's Role in Developing New Standards  
NIST is actively working on establishing standards for post-quantum cryptography, which include three algorithms designed to offer robust alternatives against quantum computing attacks. The selection process for these algorithms aims to ensure security across various applications.

### Emerging Post-Quantum Algorithms  
Recent studies have identified several promising post-quantum algorithms:
- **Lattice-based encryption:** Uses mathematical structures to create strong security.
- **Hash-based signatures:** Leverages hash functions to ensure data integrity.
- **Code-based and multivariate polynomial approaches:** Provide unique methods to counteract quantum attacks.

## Case Studies and Real-World Applications  
### Financial Sector Initiatives  
Financial institutions have begun recognizing the necessity of adopting post-quantum cryptographic frameworks to protect transactions and preserve customer trust. Strategic plans include moving towards quantum-safe standards and increasing security awareness among stakeholders.

### Telecommunications Advancements  
Research units like Toshiba have made significant strides with real-world applications, successfully transmitting quantum-encrypted messages over extensive telecommunications networks. Such advancements pave the way for secure communication protocols in a future dominated by quantum technologies.

## Conclusion  
The advent of quantum computing by 2030 will inevitably alter the landscape of cryptography. Governments and organizations must prepare for this transformation by implementing robust measures and transitioning towards post-quantum cryptographic systems. Immediate action will mitigate the risks posed by quantum threats, safeguarding sensitive digital information and ensuring the continued trust in cryptographic practices.

## Follow-Up Questions  
- What specific actions should organizations take to transition seamlessly to post-quantum cryptography?
- What are the legal implications of transitioning to new cryptographic standards?
- How can businesses measure the effectiveness of their quantum-resistant systems?  
- What specific realms of research into post-quantum algorithms are still ongoing, and what are their potential impacts?

Short Summary: By 2030, advancements in quantum computing are set to disrupt traditional cryptographic methods, particularly those reliant on prime factorization and discrete logarithms. This creates a pressing need for organizations to transition to post-quantum cryptographic systems to ensure data security against potential quantum-based attacks, with significant initiatives already underway from various national and international bodies.
Follow-up Questions: ['What specific actions should organizations take to transition seamlessly to post-quantum cryptography?', 'What are the legal implications of transitioning to new cryptographic standards?', 'How can businesses measure the effectiveness of their quantum-resistant systems?', 'What specific realms of research into post-quantum algorithms are still ongoing, and what are their potential impacts?']

Workflow execution complete. Check your WhatsApp for the report!
Remember to check the trace at https://platform.openai.com/traces for d